In [8]:
using DynamicPolynomials, SparseArrays

println("***Problem setting***")

n=5

println("Number of variable: n=",n)
println("====================")

@polyvar x[1:n]# variables

function generate_random_poly(v)
    lv=length(v)
    c=2*rand(Float64,lv).-1
    return sum(c[j]*v[j] for j in 1:lv)
end
# random quadratic objective function f
v=reverse(monomials(x,2))
f=generate_random_poly(v)


# unit sphere constraint
g=[1.0-sum(x.^2)] #type of coefficients of each polynomial must be float

m=length(g)
println("Number of inequality constraints: m=",m)
println("====================")

l=ceil(Int64, n/4)

h=Vector{Polynomial{true,Float64}}(undef,l)
randx=2*rand(n).-1# create a feasible solution
randx=rand(1)[1]*randx./sqrt(sum(randx.^2))


for j in 1:l
    h[j]=generate_random_poly(v[2:end])
    h[j]-=h[j](x => randx) #make constraints feasible
end

l=length(h)
println("Number of equality constraints: l=",l)


***Problem setting***
Number of variable: n=5
Number of inequality constraints: m=1
Number of equality constraints: l=2


In [9]:
include("../src/ctpPOP.jl")
using .ctpPOP

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=ctpPOP.get_info(x,f,g,h,sparse=false);

In [10]:
k=2

println("Relaxed order: k=",k)
println("====================")
t=1
println("Sparse order: t=",t)

Relaxed order: k=2
Sparse order: t=1


In [11]:
include("../src/ctpPOP.jl")
using .ctpPOP

opt_val=ctpPOP.POP_TS_CGAL(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,t,
                         maxit=Int64(1e6),tol=1e-3,
                         use_eqcons_to_get_constant_trace=false,
                         check_tol_each_iter=true)

  Computing constant trace status: OPTIMAL
  Constant trace: ak = 3.0
  Number of blocks: omega=

4
  Size of the largest block: s^max=16
  Number of equality trace constraints: zeta=114
Modeling time:
  0.513487 seconds (569.55 k allocations: 29.184 MiB, 8.24% gc time)
iter=1   val=-3.579565016741219   gap=2.909157583045339e-16   feas=3.703219850198637
iter=2   val=-1.1931883389137399   gap=0.49758099865176814   feas=0.9704996580190388
iter=4   val=-1.0749916451356327   gap=0.35479213130795517   feas=0.6671687973252838
iter=8   val=-1.4277675128057306   gap=0.19649271557073955   feas=0.7512318245164814
iter=16   val=-0.8209167006946732   gap=0.21652033463892342   feas=0.14644382392276067
iter=32   val=-0.7306316941642405   gap=0.1495943286500052   feas=0.07542184711834668
iter=64   val=-0.7889757964989162   gap=0.01699333608802713   feas=0.08858701110258771
iter=128   val=-0.7838386368107848   gap=0.07006859097977103   feas=0.01101626069188409
iter=256   val=-0.7983090232663628   gap=0.001130537918838376   feas=0.013992999120662387
iter=512   val=-0.7934114318204264   gap=0.054065

-0.7938012982187435

In [12]:
using TSSOS

@time tssos_first([[f];g;h],x,k,numeq=length(h),TS="block",quotient=false);

------------------------------------------------------
The sizes of blocks:
[16, 5]
[1, 1]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 86              
  Cones                  : 0               
  Scalar variables       : 34              
  Matrix variables       : 3               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  

In [13]:
include("../src/ctpPOP.jl")
using .ctpPOP

optval=ctpPOP.POP_NLP(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f)

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       48
Number of nonzeros in inequality constraint Jacobian.:        5
Number of nonzeros in Lagrangian Hessian.............:       48

Total number of variables............................:        5
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        2
Total number of inequality constraints...............:        1
        inequality constraints with only lower bounds:        1
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

0.0